In [30]:
import pandas as pd
import numpy as np
import ast

In [31]:
import sys
print(sys.executable)

/home/linux-ubuntu/neo4j_env/bin/python


In [32]:
movies = pd.read_csv('datasets/tmdb_5000_movies.csv')
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [33]:
credits = pd.read_csv('datasets/tmdb_5000_credits.csv')
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [34]:
tmdb = movies.merge(credits, on="title")
tmdb.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [35]:
tmdb = tmdb [[
    'title',
    'genres',
    'cast',
    'crew',
    'release_date'
]]
tmdb.head()

,title,genres,cast,crew,release_date
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",2007-05-19
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",2015-10-26
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",2012-07-16
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",2012-03-07


In [36]:
def get_director(crew):
    crew = ast.literal_eval(crew)
    for person in crew:
        if person ['job'] == 'Director':
            return person['name']
    return None

tmdb['director'] = tmdb['crew'].apply(get_director)
tmdb.head()

,title,genres,cast,crew,release_date,director
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10,James Cameron
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",2007-05-19,Gore Verbinski
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",2015-10-26,Sam Mendes
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",2012-07-16,Christopher Nolan
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",2012-03-07,Andrew Stanton


In [37]:
def get_cast(cast):
    cast = ast.literal_eval(cast)
    return [actor['name'] for actor in cast[:5]]

tmdb['actors'] = tmdb['cast'].apply(get_cast)
tmdb.head()

,title,genres,cast,crew,release_date,director,actors
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weave..."
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",2007-05-19,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley, ..."
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",2015-10-26,Sam Mendes,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R..."
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",2012-07-16,Christopher Nolan,"[Christian Bale, Michael Caine, Gary Oldman, A..."
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",2012-03-07,Andrew Stanton,"[Taylor Kitsch, Lynn Collins, Samantha Morton,..."


In [38]:
def extract_names(text):
    text = ast.literal_eval(text)
    return [item['name'] for item in text]

tmdb['genres'] = tmdb['genres'].apply(extract_names)
tmdb.head()

,title,genres,cast,crew,release_date,director,actors
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weave..."
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",2007-05-19,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley, ..."
2,Spectre,"[Action, Adventure, Crime]","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",2015-10-26,Sam Mendes,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R..."
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",2012-07-16,Christopher Nolan,"[Christian Bale, Michael Caine, Gary Oldman, A..."
4,John Carter,"[Action, Adventure, Science Fiction]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",2012-03-07,Andrew Stanton,"[Taylor Kitsch, Lynn Collins, Samantha Morton,..."


In [39]:
print(tmdb.columns)
tmdb.shape

Index(['title', 'genres', 'cast', 'crew', 'release_date', 'director',
       'actors'],
      dtype='str')


(4809, 7)

In [40]:
tmdb['year'] = pd.to_datetime(tmdb['release_date'], errors='coerce').dt.year
tmdb['year'] = tmdb['year'].astype('Int64')
print(list(tmdb.columns))
# tmdb.head()

['title', 'genres', 'cast', 'crew', 'release_date', 'director', 'actors', 'year']


In [41]:
tmdb = tmdb.drop(columns=['cast', 'crew', 'release_date'])
tmdb.head()

,title,genres,director,actors,year
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",2009
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",2007
2,Spectre,"[Action, Adventure, Crime]",Sam Mendes,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",2015
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Christopher Nolan,"[Christian Bale, Michael Caine, Gary Oldman, A...",2012
4,John Carter,"[Action, Adventure, Science Fiction]",Andrew Stanton,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",2012


In [42]:
imdb = pd.read_csv('datasets/imdb_top_1000.csv')
imdb.head()

,Poster_Link,title,year,Certificate,Runtime,genre,imdb_rating,Overview,Meta_score,director,actor1,actor2,actor3,actor4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [43]:
imdb.columns

Index(['Poster_Link', 'title', 'year', 'Certificate', 'Runtime', 'genre',
       'imdb_rating', 'Overview', 'Meta_score', 'director', 'actor1', 'actor2',
       'actor3', 'actor4', 'No_of_Votes', 'Gross'],
      dtype='str')

In [44]:
imdb['actors_imdb'] = imdb[['actor1', 'actor2','actor3', 'actor4']].values.tolist()
imdb['year'] = pd.to_numeric(imdb['year'], errors='coerce').astype('Int64')
imdb = imdb[['title', 'year', 'imdb_rating', 'actors_imdb']]
imdb.head()

,title,year,imdb_rating,actors_imdb
0,The Shawshank Redemption,1994,9.3,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,1972,9.2,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,The Dark Knight,2008,9.0,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,The Godfather: Part II,1974,9.0,"[Al Pacino, Robert De Niro, Robert Duvall, Dia..."
4,12 Angry Men,1957,9.0,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."


In [45]:
final_data = tmdb.merge(imdb, on=['title', 'year'], how='left')
final_data.head()

,title,genres,director,actors,year,imdb_rating,actors_imdb
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",2009,7.8,"[Sam Worthington, Zoe Saldana, Sigourney Weave..."
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",2007,NaN,NaN
2,Spectre,"[Action, Adventure, Crime]",Sam Mendes,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",2015,NaN,NaN
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Christopher Nolan,"[Christian Bale, Michael Caine, Gary Oldman, A...",2012,8.4,"[Christian Bale, Tom Hardy, Anne Hathaway, Gar..."
4,John Carter,"[Action, Adventure, Science Fiction]",Andrew Stanton,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",2012,NaN,NaN


In [46]:
final_data = final_data[[
    'title',
    'genres',
    'actors',
    'director',
    'year',
    'imdb_rating'
]]
final_data.head()
final_data.shape

(4809, 6)

In [47]:
final_data.iloc[0]


title                                                     Avatar
genres             [Action, Adventure, Fantasy, Science Fiction]
actors         [Sam Worthington, Zoe Saldana, Sigourney Weave...
director                                           James Cameron
year                                                        2009
imdb_rating                                                  7.8
Name: 0, dtype: object

In [48]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [53]:
# from neo4j import GraphDatabase
# import pandas as pd

# URI = "bolt://localhost:7687"
# USER = "neo4j"
# PASSWORD = "12345678"

# class MovieGraph:
#     def __init__(self, uri, user, password):
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()


#     def add_movies(self, movies_df):
#         with self.driver.session() as session:
#             for _, row in movies_df.iterrows():

#                 genres = row['genres'].split('|')
                
#                 session.execute_write(self._create_movie_and_genres, row['title'], row['movieId'], genres)

#     @staticmethod
#     def _create_movie_and_genres(tx, title, movie_id, genres):
#         tx.run("MERGE (m:Movie {id: $id, title: $title})", id=movie_id, title=title)
        
#         for genre_name in genres:
#             tx.run("""
#                 MERGE (g:Genre {name: $name})
#                 WITH g
#                 MATCH (m:Movie {id: $id})
#                 MERGE (m)-[:IN_GENRE]->(g)
#             """, name=genre_name, id=movie_id)


# df = pd.read_csv('datasets/movies.csv')

# app = MovieGraph(URI, USER, PASSWORD)
# print("white")
# app.add_movies(df.head(100))
# app.close()
# print("end")

In [56]:
from neo4j import GraphDatabase
import pandas as pd

URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "12345678"

class MovieGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def add_movies(self, df):
        with self.driver.session() as session:
         for _, row in df.iterrows():

            genres = row['genres'] if isinstance(row['genres'], list) else []
            actors = row['actors'] if isinstance(row['actors'], list) else []

            session.execute_write(
               self._create_full_movie,
               row['title'],
               int(row['year']),
               float(row['imdb_rating']),
               genres,
               actors,
               row['director']
            )

    @staticmethod
    def _create_full_movie(tx, title, year, rating, genres, actors, director):

        #Movie Node
        tx.run("""
           MERGE (m:Movie {title: $title})
           SET m.year = $year,
               m.rating = $rating
        """, title = title, year = year, rating = rating)
        

        #Genres
        for g in genres:
            tx.run("""
               MERGE (genre:Genre {name: $g})
               WITH genre
               MATCH (m:Movie {title: $title})
               MERGE (m)-[:HAS_GENRE]->(genre)
            """, g = g, title = title)

        #Actors
        for a in actors:
            tx.run("""
               MERGE (actor:Actor {name: $a})
               WITH actor
               MATCH (m:Movie {title: $title})
               MERGE (actor)-[:ActED_IN]->(m)
            """, a = a, title = title)

        #Director
        tx.run("""
           MERGE (d:Director {name: $director})
           WITH d
           MATCH (m:Movie {title: $title})
           MERGE (d)-[:DIRECTED]->(m)
        """, director = director, title = title)
               

In [57]:
app = MovieGraph(URI, USER, PASSWORD)

app.add_movies(final_data.head(500)) 

app.close()

print("Graph created!")


Graph created!
